In [18]:
import pandas as pd
import os
import re
import seaborn as sns
from matplotlib import pyplot as plt
import sys
import folium
from sklearn.preprocessing import MinMaxScaler

In [50]:
df = pd.read_csv("../../data/raw/acs/acs_raw_merged.csv", index_col = 0)

In [51]:
# TODO: add in house/rent values?
df.head()

,GEOID,NAME,variable,estimate,moe,acs,xmin,ymin,xmax,ymax
1,20161,"Riley County, Kansas",B02001_001,72602.0,NaN,2016-2021,-96.961693,39.043257,-96.389046,39.5665
2,20161,"Riley County, Kansas",B02001_002,58797.0,457.0,2016-2021,-96.961693,39.043257,-96.389046,39.5665
3,20161,"Riley County, Kansas",B02001_003,4509.0,302.0,2016-2021,-96.961693,39.043257,-96.389046,39.5665
4,20161,"Riley County, Kansas",B02001_004,602.0,272.0,2016-2021,-96.961693,39.043257,-96.389046,39.5665
5,20161,"Riley County, Kansas",B03002_012,6125.0,NaN,2016-2021,-96.961693,39.043257,-96.389046,39.5665


In [52]:
del df["GEOID"]

In [53]:

rename_dict_acs = {
    "B02001_001": "total_pop",
    "B02001_002": "total_white",
    "B02001_003": "total_black",
    "B02001_004": "total_american_indian_alaskan",
    "B03002_012": "total_hispanic_latino",
    "B19049_001": "median_income",
    "C17002_002": "income_to_poverty_below_0.5",
    "C17002_003": "income_to_poverty_0.5_1",
    "B19013_001": "median_houehold_income",
    "B25064_001": "median_gross_rent",
    "B25077_001": "median_house_value"
}

# replace the values
df['variable'] = df['variable'].replace(rename_dict_acs)


In [55]:
df['variable'].value_counts()

variable
total_pop                        16102
total_white                      16102
total_black                      16102
total_american_indian_alaskan    16102
total_hispanic_latino            16102
median_houehold_income           16102
median_income                    16102
median_gross_rent                16102
median_house_value               16102
income_to_poverty_below_0.5      16102
income_to_poverty_0.5_1          16102
Name: count, dtype: int64

In [56]:
# split the name column
df[['county', 'state']] = df['NAME'].str.split(',', expand=True)
df['county'] = df['county'].str.replace('County', '').str.strip()
df['state'] = df['state'].str.strip()

# drop the original column
df.drop(columns=['NAME'], inplace=True)

In [57]:
df.head()

,variable,estimate,moe,acs,xmin,ymin,xmax,ymax,county,state
1,total_pop,72602.0,NaN,2016-2021,-96.961693,39.043257,-96.389046,39.5665,Riley,Kansas
2,total_white,58797.0,457.0,2016-2021,-96.961693,39.043257,-96.389046,39.5665,Riley,Kansas
3,total_black,4509.0,302.0,2016-2021,-96.961693,39.043257,-96.389046,39.5665,Riley,Kansas
4,total_american_indian_alaskan,602.0,272.0,2016-2021,-96.961693,39.043257,-96.389046,39.5665,Riley,Kansas
5,total_hispanic_latino,6125.0,NaN,2016-2021,-96.961693,39.043257,-96.389046,39.5665,Riley,Kansas


In [58]:
df['county'].value_counts()

county
Washington                    1650
Jefferson                     1375
Franklin                      1320
Lincoln                       1265
Jackson                       1265
                              ... 
Yakutat City and Borough        55
Jefferson Parish                55
Valdez-Cordova Census Area      33
Chugach Census Area             22
Copper River Census Area        22
Name: count, Length: 1957, dtype: int64

In [59]:
df.loc[df['county'] == "Copper River Census Area",:]

,variable,estimate,moe,acs,xmin,ymin,xmax,ymax,county,state
8064,total_pop,2635.0,NaN,2016-2021,-147.7443,60.391676,-141.001894,63.217781,Copper River Census Area,Alaska
8065,total_white,1535.0,144.0,2016-2021,-147.7443,60.391676,-141.001894,63.217781,Copper River Census Area,Alaska
8066,total_black,3.0,5.0,2016-2021,-147.7443,60.391676,-141.001894,63.217781,Copper River Census Area,Alaska
8067,total_american_indian_alaskan,728.0,145.0,2016-2021,-147.7443,60.391676,-141.001894,63.217781,Copper River Census Area,Alaska
8068,total_hispanic_latino,128.0,63.0,2016-2021,-147.7443,60.391676,-141.001894,63.217781,Copper River Census Area,Alaska
8069,median_houehold_income,67439.0,11414.0,2016-2021,-147.7443,60.391676,-141.001894,63.217781,Copper River Census Area,Alaska
8070,median_income,67439.0,11414.0,2016-2021,-147.7443,60.391676,-141.001894,63.217781,Copper River Census Area,Alaska
8071,median_gross_rent,780.0,151.0,2016-2021,-147.7443,60.391676,-141.001894,63.217781,Copper River Census Area,Alaska
8072,median_house_value,232200.0,17000.0,2016-2021,-147.7443,60.391676,-141.001894,63.217781,Copper River Census Area,Alaska
8073,income_to_poverty_below_0.5,170.0,104.0,2016-2021,-147.7443,60.391676,-141.001894,63.217781,Copper River Census Area,Alaska


In [60]:
df['acs'].value_counts()

acs
2016-2021    35431
2015-2020    35431
2014-2019    35420
2013-2018    35420
2012-2017    35420
Name: count, dtype: int64

In [61]:
df.to_csv("../../data/clean/acs.csv", index=False)